# Specific arguments for particular field geometry

The openPMD format supports 3 types of geometries:
- Cartesian 2D
- Cartesian 3D
- Cylindrical with azimuthal decomposition (thetaMode)

This notebook shows how to use the arguments of `get_field` which are specific to a given geometry.

## (optional) Preparing this notebook to run it locally

If you choose to run this notebook locally, you will need to download the openPMD data files which will then be visualized. To do so, change the flag `download_data` to `True`, and execute the following cell. (Downloading the data may take a few seconds.)

In [5]:
download_data = True
if download_data:
    # Download
    !git clone https://github.com/openPMD/openPMD-example-datasets.git
    # Decompress
    !tar zxf openPMD-example-datasets/example-2d.tar.gz
    !tar zxf openPMD-example-datasets/example-3d.tar.gz
    !tar zxf openPMD-example-datasets/example-thetaMode.tar.gz

Cloning into 'openPMD-example-datasets'...
remote: Counting objects: 14, done.
remote: Total 14 (delta 1), reused 1 (delta 1), pack-reused 12
Unpacking objects: 100% (14/14), done.
Username for 'https://github.com': ^Cerror: external filter git-lfs smudge %f failed 2

tar: Error opening archive: Failed to open 'openPMD-example-datasets/example-3d.tar.gz'
tar: Error opening archive: Failed to open 'openPMD-example-datasets/example-thetaMode.tar.gz'
